In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import scale
from keras.models import Sequential
from keras.layers import Dense, Dropout

Using TensorFlow backend.


In [3]:
path_train = "radioMap.csv"
path_validation = "wifi_4th_floor_test.csv"

#Explicitly pass header=0 to be able to replace existing names 
train_df = pd.read_csv(path_train,header = 0)
train_AP_strengths = train_df.ix[:,:185] #select first 520 columns
print (train_AP_strengths)

    wifi0  wifi1  wifi2  wifi3  wifi4  wifi5  wifi6  wifi7  wifi8  wifi9  \
0     -72    -60    -67    -67    -67    -61    -69    -69    -67    -76   
1     -72    -60    -67    -67    -67    -61    -69    -69    -67    -76   
2     -68    -59    -68    -65    -69    -62    -69    -69    -67    -72   
3     -68    -59    -68    -65    -69    -62    -69    -69    -67    -72   
4     -68    -59    -68    -65    -69    -62    -69    -69    -67    -72   
5     -71    -56    -68    -66    -65    -56    -69    -69    -65    -67   
6     -71    -56    -68    -66    -65    -56    -69    -69    -65    -67   
7     -71    -56    -68    -66    -65    -56    -69    -69    -65    -67   
8     -79    -69    -71    -73    -70    -68    -80    -69    -72    -74   
9     -79    -69    -71    -73    -70    -68    -80    -69    -72    -74   
10    -79    -69    -71    -73    -70    -68    -80    -69    -72    -74   
11    -75    -69    -75    -75    -74    -63    -75    -73    -74    -74   
12    -75   

In [4]:
train_AP_features = scale(np.asarray(train_AP_strengths))
print (train_AP_features)

[[ 1.69355761  1.28632643  0.19582302 ...,  0.522225    0.         -0.94410949]
 [ 1.69355761  1.28632643  0.19582302 ...,  0.522225    0.         -0.94410949]
 [ 2.31691402  1.40082234  0.10034712 ...,  0.522225    0.         -0.96632916]
 ..., 
 [ 1.5377185   0.82834282  0.48225072 ...,  0.522225    0.          0.9001236 ]
 [ 1.5377185   0.82834282  0.48225072 ...,  0.522225    0.          0.9001236 ]
 [ 1.5377185   0.82834282  0.48225072 ...,  0.522225    0.          0.9001236 ]]


C:\Program Files\Anaconda3\lib\site-packages\sklearn\utils\validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [12]:
from keras.utils import to_categorical
train_labels = np.asarray(train_df.ix[:,185])
#train_labels_encoding = zeros((98, 98))
train_labels_one_hot_encoding = to_categorical(train_labels)
print (len(train_labels_one_hot_encoding[0]))

98


In [14]:
nb_epochs = 20
batch_size = 10
input_size = 185
num_classes = 98

print (train_AP_features.shape)

(98, 185)


In [ ]:
def encoder():
    model = Sequential()
    model.add(Dense(256, input_dim=input_size, activation='tanh', bias=True))
    model.add(Dense(128, activation='tanh', bias=True))
    model.add(Dense(64, activation='tanh', bias=True))
    return model
	
def decoder(e):   
    e.add(Dense(128, input_dim=64, activation='tanh', bias=True))
    e.add(Dense(256, activation='tanh', bias=True))
    e.add(Dense(input_size, activation='tanh', bias=True))
    e.compile(optimizer='adam', loss='mse')
    return e
	
def classifier(d):
    num_to_remove = 3
    for i in range(num_to_remove):
        d.pop()
    d.add(Dense(128, input_dim=64, activation='tanh', bias=True))
    d.add(Dense(128, activation='tanh', bias=True))
    d.add(Dense(num_classes, activation='softmax', bias=True))
    d.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
    return d